In [1]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

print(grandparent_dir)
sys.path.append(grandparent_dir)

c:\Workspace\Code\Repos\pieofcode1\ticketier


In [4]:
import os
from dotenv import load_dotenv
from pprint import pprint

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AzureAISearchAgentTool,
    PromptAgentDefinition,
    AzureAISearchToolResource,
    AISearchIndexResource,
    AzureAISearchQueryType,
)
from openai.types.responses.response_input_param import FunctionCallOutput, ResponseInputParam
from datetime import datetime

load_dotenv()

True

In [7]:
index_name = "rel-notes-index"
project_connection_id = "aisearchpiesvc"

In [8]:
# [START tool_declaration]
tool = AzureAISearchAgentTool(
    azure_ai_search=AzureAISearchToolResource(
        indexes=[
            AISearchIndexResource(
                project_connection_id=project_connection_id,
                index_name=index_name,
                query_type=AzureAISearchQueryType.SIMPLE,
            ),
        ]
    )
)
# [END tool_declaration]

In [9]:
q1 = "Describe the document"
q2 = "Extract the known issues details"
q3 = "Extract the known issues details with issue id and description in JSON format"

In [ ]:
with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"], credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):
    agent = project_client.agents.create_version(
        agent_name="KnownIssuesSearch-Agent",
        definition=PromptAgentDefinition(
            model=os.environ["GPT_MODEL_DEPLOYMENT_NAME"],
            instructions="""You are a helpful assistant. You must always provide citations for
            answers using the tool and render them as: `\u3010message_idx:search_idx\u2020source\u3011`.""",
            tools=[tool],
        ),
        description="You are a helpful agent.",
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

    user_input = ""
    while not user_input.strip():
        user_input = input("Enter your question (e.g., 'Extract the known issues details'): \n")

        stream_response = openai_client.responses.create(
            stream=True,
            tool_choice="required",
            input=user_input,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        )

        for event in stream_response:
            if event.type == "response.created":
                print(f"Follow-up response created with ID: {event.response.id}")
            elif event.type == "response.output_text.delta":
                print(f"Delta: {event.delta}")
            elif event.type == "response.text.done":
                print(f"\nFollow-up response done!")
            elif event.type == "response.output_item.done":
                if event.item.type == "message":
                    item = event.item
                    if item.content[-1].type == "output_text":
                        text_content = item.content[-1]
                        for annotation in text_content.annotations:
                            if annotation.type == "url_citation":
                                print(
                                    f"URL Citation: {annotation.url}, "
                                    f"Start index: {annotation.start_index}, "
                                    f"End index: {annotation.end_index}"
                                )
            elif event.type == "response.completed":
                print(f"\nFollow-up completed!")
                print(f"Full response: {event.response.output_text}")

        
    # print("\nCleaning up...")
    # project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
    # print("Agent deleted")

Agent created (id: KnownIssuesSearch-Agent:1, name: KnownIssuesSearch-Agent, version: 1)
Follow-up response created with ID: resp_06a3743a34d0abef00691e0b65e3148196ae760563e54eb27f
Delta: Here
Delta:  are
Delta:  the
Delta:  known
Delta:  issues
Delta:  extracted
Delta: :


Delta: For
Delta:  Release
Delta:  
Delta: 8
Delta: .
Delta: 1
Delta: p
Delta: 11
Delta: s
Delta: 1
Delta: :

Delta: -
Delta:  Devices
Delta:  reboot
Delta:  repeatedly
Delta:  due
Delta:  to
Delta:  WAN
Delta:  Link
Delta:  Heal
Delta:  after
Delta:  firmware
Delta:  update
Delta: 

Delta: -
Delta:  Wi
Delta: -Fi
Delta:  connectivity
Delta:  issues
Delta:  after
Delta:  gateway
Delta:  firmware
Delta:  update
Delta: 

Delta: -
Delta:  Logging
Delta:  improvements
Delta:  needed
Delta:  to
Delta:  diagnose
Delta:  Wi
Delta: -Fi
Delta:  database
Delta:  corruption
Delta:  issues
Delta:  following
Delta:  device
Delta:  re
Delta: boots
Delta:  across
Delta:  different
Delta:  models
Delta: 

Delta: -
Delta:  After
Del